# Notebook for capstone project

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import lxml.html as lh
import requests
import folium

In [31]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [41]:
!pip install folium

     |████████████████████████████████| 92kB 6.6MB/s eta 0:00:011


In [32]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')


In [33]:


#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    
    col.append((name,[]))
    

In [34]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [35]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
# remove \n -new line characters from the df
df = df.replace('\n','', regex=True)
#if Borough is Not assigned then drop the row
df.drop(df.loc[df['Borough\n']=='Not assigned'].index, inplace=True)
#if Postal code is Not empty then drop the row
df.drop(df.loc[df['Postal code\n']==''].index, inplace=True)
# set the index as Borough because after the drop index is not serial
df.set_index('Postal code\n',inplace=True)
#rest the index column as a seriel numbers 
df.reset_index(inplace=True)
#replace / in Neighborhood to comma.
df['Neighborhood\n']=df['Neighborhood\n'].str.replace("/",",")






In [21]:
#Click here and press Shift+Enter
!wget -O cell_samples.csv https://cocl.us/Geospatial_data

In [36]:
df1 = pd.read_csv("cell_samples.csv")
df1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
#Merge 2 data frame that matches with the postcodes from both the data frames.
df = pd.merge(left=df, right=df1, how="left", left_on=["Postal code\n"], right_on=["Postal Code"])
df = df.drop("Postal Code", axis=1)


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [38]:
from geopy.geocoders import Nominatim 
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto  are 43.6534817, -79.3839347.


In [44]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough\n'], df['Neighborhood\n']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork